In [1]:
!pip install semantic-kernel


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Semantic Kernel orchestation demo for multiple step plans

This demo is meant to provide insights on how the native functions can be used as plugins for semantic kernel.
There are two plugins:
    - ServiceDesk: A ticketing service plugin useful to retrieve information about incidences.
    - Invoices DB: A DB plugin to retrieve and execute read/write operations on invoices.

The questions used are the following:
    - lista las incidencias wifi
    - lista todas las facturas del usuario con id 5
    - Actualiza la siguiente factura: {id=1, test=True, user_id=5}

Each input should use a different function.

The orchestation is performed by semantic kernel by leveraging the loaded plugins and a Custom Planner is used in order to determine the steps required to answer a given question. After that the plans generated are executed.

### imports

In [2]:
import os
from typing import List

import semantic_kernel as sk
from utils.custom_planner import CustomBasicPlanner
from utils.input_model import Question
from semantic_kernel.planners import BasicPlanner

from semantic_kernel.planners.basic_planner import Plan
import utils.sk_utils as sk_utils

### Generate dummy data in Question format
Generate a variety of questions requiring from 1 to 2 steps in a plan.

In [3]:
def get_dummy_input() -> List[Question]:
    json_data = {
        "user_id": 7,
        "message_id": 7,
        "chat_id": 4,
        "domain_id": 0,
        "question": "lista las incidencias wifi",
        "plugins": [{
            "name": "ServiceDesk",
            "url": "http://localhost:9001/query",
            "configuration":{
                "url": "https://cosuitg.sothis.tech/api/v3",
                "token": "67325F3F-A481-44B7-8AEA-C69AF3337882"
            }
        }]
    }

    # Create an instance of the Question model using the JSON data
    questions = [Question(**json_data)]
    
    json_data["question"] = "lista todas las facturas del usuario con id 5"
    questions.append(Question(**json_data))

    json_data["question"] = "Actualiza la siguiente factura: {id=1, test=True, user_id=5}"
    questions.append(Question(**json_data))

    json_data["question"] = "General question"
    questions.append(Question(**json_data))

    json_data["question"] = "Tell me some facts about the most populated city in the world."
    questions.append(Question(**json_data))

    return questions

### Instantiate the kernel, load the plugins and create the planner

In [4]:
# Initialize the kernel
kernel = sk_utils.get_kernel_router()

# Import plugins
plugins = sk_utils.load_plugins(kernel=kernel)

planner = CustomBasicPlanner(service_id="planner")# BasicPlanner(service_id="planner")


### Create and run questions
Run simple questions to access one single plugin. The last question requires 2 plugins so it's a two step plan.

In [5]:
questions = get_dummy_input()

for question in questions:
    # Create a plan
    # plan: Plan = await planner.create_plan(question.question)
    plan: Plan = await planner.create_plan(question.question, kernel=kernel)

    # modify object passed 
    # Execute the plan
    result = await planner.execute_plan(plan, kernel, question)
    print("-------------------------------\n\n")
    print(f"Question: {question.question}")
    print("Plan results:")
    print(result)
    print("-------------------------------\n\n")

Overwriting function "CreatePlan" in collection


-------------------------------


Question: lista las incidencias wifi
Plan results:
requested list of incidences with question: user_id=7 message_id=7 chat_id=4 domain_id=0 question='lista las incidencias wifi' plugins=[Plugin(name='ServiceDesk', url='http://localhost:9001/query', configuration={'url': 'https://cosuitg.sothis.tech/api/v3', 'token': '67325F3F-A481-44B7-8AEA-C69AF3337882'})]
-------------------------------




Overwriting function "CreatePlan" in collection


-------------------------------


Question: lista todas las facturas del usuario con id 5
Plan results:
requested invoices with question: user_id=7 message_id=7 chat_id=4 domain_id=0 question='lista todas las facturas del usuario con id 5' plugins=[Plugin(name='ServiceDesk', url='http://localhost:9001/query', configuration={'url': 'https://cosuitg.sothis.tech/api/v3', 'token': '67325F3F-A481-44B7-8AEA-C69AF3337882'})]
-------------------------------




Overwriting function "CreatePlan" in collection


-------------------------------


Question: Actualiza la siguiente factura: {id=1, test=True, user_id=5}
Plan results:
requested write operations on invoices with question: user_id=7 message_id=7 chat_id=4 domain_id=0 question='Actualiza la siguiente factura: {id=1, test=True, user_id=5}' plugins=[Plugin(name='ServiceDesk', url='http://localhost:9001/query', configuration={'url': 'https://cosuitg.sothis.tech/api/v3', 'token': '67325F3F-A481-44B7-8AEA-C69AF3337882'})]
-------------------------------




Overwriting function "CreatePlan" in collection


-------------------------------


Question: General question
Plan results:
requested rag with question: user_id=7 message_id=7 chat_id=4 domain_id=0 question='General question' plugins=[Plugin(name='ServiceDesk', url='http://localhost:9001/query', configuration={'url': 'https://cosuitg.sothis.tech/api/v3', 'token': '67325F3F-A481-44B7-8AEA-C69AF3337882'})] It is the capital and largest city of the autonomous community of Catalonia
-------------------------------


requesting cities with filter {'population': 'max(population)'}
-------------------------------


Question: Tell me some facts about the most populated city in the world.
Plan results:
requested rag with question: user_id=7 message_id=7 chat_id=4 domain_id=0 question='Tell me some facts about the most populated city in the world.' plugins=[Plugin(name='ServiceDesk', url='http://localhost:9001/query', configuration={'url': 'https://cosuitg.sothis.tech/api/v3', 'token': '67325F3F-A481-44B7-8AEA-C69AF3337882'})] It is the capita